In [1]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
# 创建默认的InteractiveSession，这样后面执行各项操作就无须指定Session
sess = tf.InteractiveSession()

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
in_units = 784 # 输入节点数
h1_units = 300 # 隐含层的输出节点数
W1 = tf.Variable(tf.truncated_normal([in_units, h1_units], stddev=0.1)) # 隐含层的权重 初始化为截断的正态分布，其标准差为0.1
b1 = tf.Variable(tf.zeros([h1_units]))    # 隐含层的偏置 赋值为0
W2 = tf.Variable(tf.zeros([h1_units, 10]))
b2 = tf.Variable(tf.zeros([10]))

In [3]:
x = tf.placeholder(tf.float32, [None, in_units])
# 在训练和预测时，Dropout的比率keep_prob(保留节点的概率)是不一样的。
# 通常在训练时小于 1，在预测时等于 1
# 所以也把Dropout的比率作为计算图的输入，并定义成一个placeholder
keep_prob = tf.placeholder(tf.float32)

In [6]:
# 实现一个激活函数为ReLU的隐含层
# y = relu(W1*x + b1)
hidden1 = tf.nn.relu(tf.matmul(x, W1) + b1)
# 实现Dropout功能，随机将一部分节点置为0
# keep_prob参数即为保留数据而不置为0的比例
hidden1_drop = tf.nn.dropout(hidden1, keep_prob)
# 输出层
y = tf.nn.softmax(tf.matmul(hidden1_drop, W2) + b2)

In [7]:
# 真实结果
y_ = tf.placeholder(tf.float32, [None, 10])
# 交叉信息熵
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))
# 选择优化器来优化loss
train_step = tf.train.AdagradOptimizer(0.3).minimize(cross_entropy)

In [8]:
tf.global_variables_initializer().run()
# 共采用3000个batch，每个batch包含100条样本
for i in range(3000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    train_step.run({x: batch_xs, y_: batch_ys, keep_prob: 0.75})

In [9]:
# argmax()返回最大值的索引 axis=1表示横轴方向上
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
# tf.cast 将True、False转化成对应的1.0、0，或者相反
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(accuracy.eval({x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

0.9796
